In [ ]:
%load_ext autoreload
%autoreload 2

from jenga.tasks.openml import OpenMLTask
from jenga.cleaning.cleaners import AutoGluonCleaner, SklearnCleaner
from jenga.corruptions.numerical import Scaling
from jenga.corruptions.generic import MissingValues
from jenga.evaluation.autocleaning import AutoCleaningEvaluator
import numpy as np

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd
X, y = load_iris( return_X_y=True)
df = pd.DataFrame(X)
df['label'] = y
df

In [ ]:
from jenga.corruptions.generic import CategoricalShift
from sklearn.model_selection import train_test_split

df['label'] = df['label'].astype(str)
train_df, test_df = train_test_split(df.copy(deep=True), test_size=.5)

df_corrupted = CategoricalShift(column='label', fraction=.5, sampling='MNAR').transform(test_df)
# df_corrupted[df_corrupted['label']!=test_df['label']]
# df_corrupted['label'] = np.nann 

df_corrupted['label_true'] = test_df['label']

df_corrupted

In [ ]:
sklearn_cleaner = SklearnCleaner(numeric_columns=[0,1,2,3], 
                           categorical_columns=['label'],
                                categorical_precision_threshold=.9).fit(train_df)

# df_outliers_removed = sklearn_cleaner.impute(df_corrupted, columns=['label'])
df_outliers_removed = sklearn_cleaner.remove_outliers(df_corrupted, columns=['label'])
df_outliers_removed

In [ ]:
debug



In [ ]:
# cleaner.fit(task.train_data)
# evaluator = AutoCleaningEvaluator(task, cleaner)

In [ ]:
corruptions = []
for impacted_column in task.numerical_columns + task.categorical_columns:
    for fraction in [0.99, 0.5, 0.25, 0.1, 0.01]:
        if impacted_column in task.numerical_columns:
            corruptions.append(Scaling(impacted_column, fraction))
        for missingness in ['MCAR', 'MAR', 'MNAR']:
            corruption = MissingValues(impacted_column, fraction, missingness=missingness, na_value=np.nan)
            corruptions.append(corruption)

In [ ]:
task = OpenMLTask(0, openml_id=42261)
model = task.fit_baseline_model(task.train_data, task.train_labels)
sklearn_cleaner = SklearnCleaner(numeric_columns=task.numerical_columns, 
                           categorical_columns=task.categorical_columns)
# cleaner = AutoGluonCleaner(numeric_columns=task.numerical_columns, 
#                            categorical_columns=task.categorical_columns)

In [ ]:
sklearn_cleaner.fit(task.train_data)
sklearn_cleaner_evaluator = AutoCleaningEvaluator(task, sklearn_cleaner)

In [ ]:
sklearn_cleaner_results = sklearn_cleaner_evaluator.evaluate(model, 5, *corruptions[:2])

In [ ]:
sklearn_cleaner_results[1].__dict__